# 📊 Análise Exploratória - Insights Iniciais

### Objetivo
Analisar a qualidade dos dados do dataset e identificar possíveis problemas para tratamento.

## Informações Gerais

### Importação do PySpark

In [2]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, sum, when, trim

### Iniciação do SparkSession

In [3]:
spark = SparkSession.builder.appName("AnaliseUber").getOrCreate()

Using Spark's default log4j profile: org/apache/spark/log4j2-defaults.properties
25/09/26 12:15:58 WARN Utils: Your hostname, codespaces-ef9f50, resolves to a loopback address: 127.0.0.1; using 10.0.12.62 instead (on interface eth0)
25/09/26 12:15:58 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Using Spark's default log4j profile: org/apache/spark/log4j2-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/09/26 12:15:59 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
25/09/26 12:15:59 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


### Leitura do CSV

In [4]:
df = spark.read.csv("ncr_ride_bookings.csv", header=True, inferSchema=True)

### Tamanho do dataset

In [5]:
num_linhas = df.count()
num_colunas = len(df.columns)
print(f"Número de linhas: {num_linhas}\nNúmero de colunas: {num_colunas}")

Número de linhas: 150000
Número de colunas: 21


### Tipos de dados

In [ ]:
for field in df.schema.fields:
    print(f"{field.name:<33} | {str(field.dataType):<15} | nullable={field.nullable}")


Date                              | DateType()      | nullable=True
Time                              | TimestampType() | nullable=True
Booking ID                        | StringType()    | nullable=True
Booking Status                    | StringType()    | nullable=True
Customer ID                       | StringType()    | nullable=True
Vehicle Type                      | StringType()    | nullable=True
Pickup Location                   | StringType()    | nullable=True
Drop Location                     | StringType()    | nullable=True
Avg VTAT                          | StringType()    | nullable=True
Avg CTAT                          | StringType()    | nullable=True
Cancelled Rides by Customer       | StringType()    | nullable=True
Reason for cancelling by Customer | StringType()    | nullable=True
Cancelled Rides by Driver         | StringType()    | nullable=True
Driver Cancellation Reason        | StringType()    | nullable=True
Incomplete Rides                  | StringType()

### Estatísticas Descritivas

In [33]:
df.describe().show()

+-------+----------------+--------------------+----------------+------------+---------------+-------------+-----------------+-----------------+---------------------------+---------------------------------+-------------------------+--------------------------+----------------+-----------------------+------------------+------------------+-------------------+-------------------+--------------+
|summary|      Booking ID|      Booking Status|     Customer ID|Vehicle Type|Pickup Location|Drop Location|         Avg VTAT|         Avg CTAT|Cancelled Rides by Customer|Reason for cancelling by Customer|Cancelled Rides by Driver|Driver Cancellation Reason|Incomplete Rides|Incomplete Rides Reason|     Booking Value|     Ride Distance|     Driver Ratings|    Customer Rating|Payment Method|
+-------+----------------+--------------------+----------------+------------+---------------+-------------+-----------------+-----------------+---------------------------+---------------------------------+---------

### Valores Ausentes

In [30]:
df.select([
    sum(
        when(
            (trim(col(c)) == "") | 
            (trim(col(c)) == "null") | 
            (trim(col(c)) == "NA") | 
            (trim(col(c)) == "NaN"), 
            1
        ).otherwise(0)
    ).alias(c) for c in df.columns
]).show()


+----+----+----------+--------------+-----------+------------+---------------+-------------+--------+--------+---------------------------+---------------------------------+-------------------------+--------------------------+----------------+-----------------------+-------------+-------------+--------------+---------------+--------------+
|Date|Time|Booking ID|Booking Status|Customer ID|Vehicle Type|Pickup Location|Drop Location|Avg VTAT|Avg CTAT|Cancelled Rides by Customer|Reason for cancelling by Customer|Cancelled Rides by Driver|Driver Cancellation Reason|Incomplete Rides|Incomplete Rides Reason|Booking Value|Ride Distance|Driver Ratings|Customer Rating|Payment Method|
+----+----+----------+--------------+-----------+------------+---------------+-------------+--------+--------+---------------------------+---------------------------------+-------------------------+--------------------------+----------------+-----------------------+-------------+-------------+--------------+---------

### Colunas com Maior Percentual de Nulos

In [37]:
# 1. Total de linhas no DataFrame
total = df.count()

null_counts = df.select([
    sum(
        when(
            (trim(col(c)) == "") | 
            (trim(col(c)) == "null") | 
            (trim(col(c)) == "NA") | 
            (trim(col(c)) == "NaN"), 
            1
        ).otherwise(0)
    ).alias(c) for c in df.columns
]).collect()[0].asDict()

# Cria lista com colunas, contagem e porcentagem
cols_nulls = [(col_name, count, 100 * count / total) for col_name, count in null_counts.items()]

# Ordena pelo número de nulos (decrescente)
cols_nulls_sorted = sorted(cols_nulls, key=lambda x: x[1], reverse=True)

# Mostrar resultado
print("Coluna                            | # Nulos | % Nulos")
for col_name, count, pct in cols_nulls_sorted:
    print(f"{col_name:<33} | {count:<7} | {pct:.2f}%")

Coluna                            | # Nulos | % Nulos
Incomplete Rides                  | 141000  | 94.00%
Incomplete Rides Reason           | 141000  | 94.00%
Cancelled Rides by Customer       | 139500  | 93.00%
Reason for cancelling by Customer | 139500  | 93.00%
Cancelled Rides by Driver         | 123000  | 82.00%
Driver Cancellation Reason        | 123000  | 82.00%
Driver Ratings                    | 57000   | 38.00%
Customer Rating                   | 57000   | 38.00%
Avg CTAT                          | 48000   | 32.00%
Booking Value                     | 48000   | 32.00%
Ride Distance                     | 48000   | 32.00%
Payment Method                    | 48000   | 32.00%
Avg VTAT                          | 10500   | 7.00%
Date                              | 0       | 0.00%
Time                              | 0       | 0.00%
Booking ID                        | 0       | 0.00%
Booking Status                    | 0       | 0.00%
Customer ID                       | 0       | 0.00

## Análise Geral

### Estrutura:
- Várias colunas numéricas armazenadas como StringType(), o que sugere a necessidade de conversão para tipos numéricos.
- Datas e horários: Date e Time já estão em DateType/TimestampType, o que é positivo.
- IDs (Booking ID, Customer ID): aparecem entre aspas triplas ("""CNR1000037"""), o que sugere problemas de ingestão de dados (aspas extras). Necessário limpar formatação.
- Campos categóricos (Booking Status, Vehicle Type, Pickup/Drop Location): precisam ser normalizados (case, espaços, valores duplicados com grafia diferente).
- Colunas numéricas salvas como string precisam ser convertidas e além disso, pode haver valores inválidos (ex.: "null", "NA", strings vazias).

### Valores Nulos:
- Colunas com altíssimo percentual de nulos